In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


SINGLE TASK LSTM LOCOMOTIONS

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_st():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    l_output = Dense(1, activation='linear', name='l_output')(shared_mlp)


    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[l_output])
    return model


# Compile the model
model = create_model_st()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse'],
              metrics=['mse','mae'])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[locomotions_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        [locomotions_test])


# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
l_loss = results[1]  # Loss for 'l_output'

# Assuming correct order, retrieve MAE values
l_mae = results[2]  # MAE for 'l_output'


print(f"\nResultados de la evaluación ORIGINAL:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para l (Regresión): {l_loss:.4f}\n"
      f"MAE para l (Regresión): {l_mae:.4f}\n"
      )


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, locomotions_train,
    sequences_test, auxiliares_test, locomotions_test,
    num_ejecuciones=1000
):
    l_maes = []


    for _ in range(num_ejecuciones):
        indices_bootstrap = resample(np.arange(len(sequences_test)), replace=True, n_samples=len(sequences_test))

        sequences_bootstrap = sequences_test[indices_bootstrap]
        auxiliares_bootstrap = auxiliares_test[indices_bootstrap]
        locomotions_bootstrap = locomotions_test[indices_bootstrap]


        # 2. Evaluate the model on the bootstrap sample (no training in each iteration)
        #predictions = model.predict([sequences_bootstrap, auxiliares_bootstrap])

         # Evaluate the model on the bootstrap sample
        results1 = model.evaluate(
        [sequences_bootstrap, auxiliares_bootstrap],
        [locomotions_bootstrap],
        verbose=0  # Set verbose to 0 to suppress output during evaluation
        )

        # Extract MAE values from the results
        l_mae = results1[2]  # MAE for 'l_output' (assuming the correct index)


        # Calcula el MAE para cada variable objetivo usando las predicciones (CÓDIGO NUEVO)
       # l_mae = mean_absolute_error(locomotions_bootstrap, predictions[0])
        #s_mae = mean_absolute_error(sensories_bootstrap, predictions[1])
        #v_mae = mean_absolute_error(vitalities_bootstrap, predictions[2])
        #p_mae = mean_absolute_error(psichologicals_bootstrap, predictions[3])
        #c_mae = mean_absolute_error(cognitions_bootstrap, predictions[4])

        l_maes.append(l_mae)


    # Función para calcular estadísticas e intervalos de confianza (CÓDIGO NUEVO)
    def calculate_ci(maes):
        mae_promedio = np.mean(maes)
        desviacion_estandar = np.std(maes)
        valor_critico = 1.96  # Para un intervalo de confianza del 95%
        margen_de_error = valor_critico * (desviacion_estandar / np.sqrt(num_ejecuciones))
        limite_inferior = mae_promedio - margen_de_error
        limite_superior = mae_promedio + margen_de_error
        return mae_promedio, (limite_inferior, limite_superior)

    l_results = calculate_ci(l_maes)


    return {
        'l': l_results
    }

# Calcula intervalo de confianza con Bootstrap (CÓDIGO NUEVO)
resultados_bootstrap = calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, locomotions_train,
    sequences_test, auxiliares_test, locomotions_test,
    num_ejecuciones=1000
)

# Imprime los resultados del Bootstrap (CÓDIGO NUEVO)
for target, (mae_promedio, intervalo_confianza) in resultados_bootstrap.items():
    print(f"MAE para {target} (Regresión): {mae_promedio:.4f} (IC 95%: {intervalo_confianza})")


<ipython-input-2-6eca49ca8491>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-2-6eca49ca8491>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 24)             │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 27)             │              0 │ dropout[0][0],         │
│                           │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 8)              │            224 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 8)              │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ l_output (Dense)          │ (None, 1)              │              9 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,977 (15.54 KB)

 Trainable params: 3,977 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.2298 - mae: 0.4170 - mse: 0.2298
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0985 - mae: 0.2386 - mse: 0.0985
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1075 - mae: 0.2480 - mse: 0.1075
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0928 - mae: 0.2300 - mse: 0.0928
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0905 - mae: 0.2336 - mse: 0.0905 
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0888 - mae: 0.2251 - mse: 0.0888
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0893 - mae: 0.2247 - mse: 0.0893
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0801 - mae: 0.2133 - mse: 0.0801 
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0801 - mae: 0.2164 - mse: 0.0801 
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0728 - mae: 0.2040 - mse: 0.0728 
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0778 - mae: 0.2116 

SINGLE TASK LSTM COGNITION

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_st():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    c_output = Dense(1, activation='linear', name='c_output')(shared_mlp)


    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[c_output])
    return model


# Compile the model
model = create_model_st()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse'],
              metrics=['mse','mae'])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[cognitions_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        [cognitions_test])


# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
c_loss = results[1]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
c_mae = results[2]  # MAE for 'c_output'


print(f"\nResultados de la evaluación ORIGINAL:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para c (Regresión): {c_loss:.4f}\n"
      f"MAE para c (Regresión): {c_mae:.4f}\n"
      )


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, cognitions_train,
    sequences_test, auxiliares_test, cognitions_test,
    num_ejecuciones=1000
):
    c_maes = []


    for _ in range(num_ejecuciones):
        indices_bootstrap = resample(np.arange(len(sequences_test)), replace=True, n_samples=len(sequences_test))

        sequences_bootstrap = sequences_test[indices_bootstrap]
        auxiliares_bootstrap = auxiliares_test[indices_bootstrap]
        cognitions_bootstrap = cognitions_test[indices_bootstrap]


        # 2. Evaluate the model on the bootstrap sample (no training in each iteration)
        #predictions = model.predict([sequences_bootstrap, auxiliares_bootstrap])

         # Evaluate the model on the bootstrap sample
        results1 = model.evaluate(
        [sequences_bootstrap, auxiliares_bootstrap],
        [cognitions_bootstrap],
        verbose=0  # Set verbose to 0 to suppress output during evaluation
        )

        # Extract MAE values from the results
        c_mae = results1[2]  # MAE for 'c_output' (assuming the correct index)


        # Calcula el MAE para cada variable objetivo usando las predicciones (CÓDIGO NUEVO)
       # l_mae = mean_absolute_error(locomotions_bootstrap, predictions[0])
        #s_mae = mean_absolute_error(sensories_bootstrap, predictions[1])
        #v_mae = mean_absolute_error(vitalities_bootstrap, predictions[2])
        #p_mae = mean_absolute_error(psichologicals_bootstrap, predictions[3])
        #c_mae = mean_absolute_error(cognitions_bootstrap, predictions[4])

        c_maes.append(c_mae)


    # Función para calcular estadísticas e intervalos de confianza (CÓDIGO NUEVO)
    def calculate_ci(maes):
        mae_promedio = np.mean(maes)
        desviacion_estandar = np.std(maes)
        valor_critico = 1.96  # Para un intervalo de confianza del 95%
        margen_de_error = valor_critico * (desviacion_estandar / np.sqrt(num_ejecuciones))
        limite_inferior = mae_promedio - margen_de_error
        limite_superior = mae_promedio + margen_de_error
        return mae_promedio, (limite_inferior, limite_superior)

    c_results = calculate_ci(c_maes)


    return {
        'c': c_results
    }

# Calcula intervalo de confianza con Bootstrap (CÓDIGO NUEVO)
resultados_bootstrap = calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, cognitions_train,
    sequences_test, auxiliares_test, cognitions_test,
    num_ejecuciones=1000
)

# Imprime los resultados del Bootstrap (CÓDIGO NUEVO)
for target, (mae_promedio, intervalo_confianza) in resultados_bootstrap.items():
    print(f"MAE para {target} (Regresión): {mae_promedio:.4f} (IC 95%: {intervalo_confianza})")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-13-63e68fc0b8b7>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-13-63e68fc0b8b7>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_11 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 24)             │              0 │ lstm_11[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 27)             │              0 │ dropout_19[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 8)              │            224 │ concatenate_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 8)              │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ c_output (Dense)          │ (None, 1)              │              9 │ dropout_20[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,977 (15.54 KB)

 Trainable params: 3,977 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.3582 - mae: 0.5061 - mse: 0.3582
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1314 - mae: 0.2943 - mse: 0.1314
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1021 - mae: 0.2588 - mse: 0.1021
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0889 - mae: 0.2451 - mse: 0.0889
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0710 - mae: 0.2160 - mse: 0.0710
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0691 - mae: 0.2092 - mse: 0.0691
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0591 - mae: 0.1918 - mse: 0.0591
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0565 - mae: 0.1886 - mse: 0.0565
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0504 - mae: 0.1750 - mse: 0.0504
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0508 - mae: 0.1733 - mse: 0.0508
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0499 - mae: 0.1741 

SINGLE TASK LSTM VITALITY

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_st():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    v_output = Dense(1, activation='linear', name='v_output')(shared_mlp)


    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[v_output])
    return model


# Compile the model
model = create_model_st()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse'],
              metrics=['mse','mae'])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[vitalities_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        [vitalities_test])


# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
v_loss = results[1]  # Loss for 'c_output'

# Assuming correct order, retrieve MAE values
v_mae = results[2]  # MAE for 'c_output'


print(f"\nResultados de la evaluación ORIGINAL:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para v (Regresión): {v_loss:.4f}\n"
      f"MAE para v (Regresión): {v_mae:.4f}\n"
      )


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, vitalities_train,
    sequences_test, auxiliares_test, vitalities_test,
    num_ejecuciones=1000
):
    v_maes = []


    for _ in range(num_ejecuciones):
        indices_bootstrap = resample(np.arange(len(sequences_test)), replace=True, n_samples=len(sequences_test))

        sequences_bootstrap = sequences_test[indices_bootstrap]
        auxiliares_bootstrap = auxiliares_test[indices_bootstrap]
        vitalities_bootstrap = vitalities_test[indices_bootstrap]


        # 2. Evaluate the model on the bootstrap sample (no training in each iteration)
        #predictions = model.predict([sequences_bootstrap, auxiliares_bootstrap])

         # Evaluate the model on the bootstrap sample
        results1 = model.evaluate(
        [sequences_bootstrap, auxiliares_bootstrap],
        [vitalities_bootstrap],
        verbose=0  # Set verbose to 0 to suppress output during evaluation
        )

        # Extract MAE values from the results
        v_mae = results1[2]  # MAE for 'v_output' (assuming the correct index)


        # Calcula el MAE para cada variable objetivo usando las predicciones (CÓDIGO NUEVO)
       # l_mae = mean_absolute_error(locomotions_bootstrap, predictions[0])
        #s_mae = mean_absolute_error(sensories_bootstrap, predictions[1])
        #v_mae = mean_absolute_error(vitalities_bootstrap, predictions[2])
        #p_mae = mean_absolute_error(psichologicals_bootstrap, predictions[3])
        #c_mae = mean_absolute_error(cognitions_bootstrap, predictions[4])

        v_maes.append(v_mae)


    # Función para calcular estadísticas e intervalos de confianza (CÓDIGO NUEVO)
    def calculate_ci(maes):
        mae_promedio = np.mean(maes)
        desviacion_estandar = np.std(maes)
        valor_critico = 1.96  # Para un intervalo de confianza del 95%
        margen_de_error = valor_critico * (desviacion_estandar / np.sqrt(num_ejecuciones))
        limite_inferior = mae_promedio - margen_de_error
        limite_superior = mae_promedio + margen_de_error
        return mae_promedio, (limite_inferior, limite_superior)

    v_results = calculate_ci(v_maes)


    return {
        'v': v_results
    }

# Calcula intervalo de confianza con Bootstrap (CÓDIGO NUEVO)
resultados_bootstrap = calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, vitalities_train,
    sequences_test, auxiliares_test, vitalities_test,
    num_ejecuciones=1000
)

# Imprime los resultados del Bootstrap (CÓDIGO NUEVO)
for target, (mae_promedio, intervalo_confianza) in resultados_bootstrap.items():
    print(f"MAE para {target} (Regresión): {mae_promedio:.4f} (IC 95%: {intervalo_confianza})")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-16-dc53a547fc74>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-16-dc53a547fc74>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_14 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 24)             │              0 │ lstm_14[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_14            │ (None, 27)             │              0 │ dropout_25[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 8)              │            224 │ concatenate_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_26 (Dropout)      │ (None, 8)              │              0 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ v_output (Dense)          │ (None, 1)              │              9 │ dropout_26[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,977 (15.54 KB)

 Trainable params: 3,977 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 1.6994 - mae: 1.2627 - mse: 1.6994
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.7993 - mae: 0.8315 - mse: 0.7993
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2462 - mae: 0.4144 - mse: 0.2462
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1195 - mae: 0.2822 - mse: 0.1195
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1295 - mae: 0.2880 - mse: 0.1295
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1075 - mae: 0.2675 - mse: 0.1075
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0939 - mae: 0.2490 - mse: 0.0939
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0924 - mae: 0.2436 - mse: 0.0924
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0772 - mae: 0.2214 - mse: 0.0772
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0785 - mae: 0.2247 - mse: 0.0785
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0745 - mae: 0.2190 

SINGLE TASK LSTM PSICHOLOGICALS

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_st():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    p_output = Dense(1, activation='linear', name='p_output')(shared_mlp)


    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[p_output])
    return model


# Compile the model
model = create_model_st()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse'],
              metrics=['mse','mae'])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[psichologicals_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        [psichologicals_test])


# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
p_loss = results[1]  # Loss for 'p_output'

# Assuming correct order, retrieve MAE values
p_mae = results[2]  # MAE for 'p_output'


print(f"\nResultados de la evaluación ORIGINAL:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para p (Regresión): {p_loss:.4f}\n"
      f"MAE para p (Regresión): {p_mae:.4f}\n"
      )


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, psichologicals_train,
    sequences_test, auxiliares_test, psichologicals_test,
    num_ejecuciones=1000
):
    p_maes = []


    for _ in range(num_ejecuciones):
        indices_bootstrap = resample(np.arange(len(sequences_test)), replace=True, n_samples=len(sequences_test))

        sequences_bootstrap = sequences_test[indices_bootstrap]
        auxiliares_bootstrap = auxiliares_test[indices_bootstrap]
        psichologicals_bootstrap = psichologicals_test[indices_bootstrap]


        # 2. Evaluate the model on the bootstrap sample (no training in each iteration)
        #predictions = model.predict([sequences_bootstrap, auxiliares_bootstrap])

         # Evaluate the model on the bootstrap sample
        results1 = model.evaluate(
        [sequences_bootstrap, auxiliares_bootstrap],
        [psichologicals_bootstrap],
        verbose=0  # Set verbose to 0 to suppress output during evaluation
        )

        # Extract MAE values from the results
        p_mae = results1[2]  # MAE for 'v_output' (assuming the correct index)


        # Calcula el MAE para cada variable objetivo usando las predicciones (CÓDIGO NUEVO)
       # l_mae = mean_absolute_error(locomotions_bootstrap, predictions[0])
        #s_mae = mean_absolute_error(sensories_bootstrap, predictions[1])
        #v_mae = mean_absolute_error(vitalities_bootstrap, predictions[2])
        #p_mae = mean_absolute_error(psichologicals_bootstrap, predictions[3])
        #c_mae = mean_absolute_error(cognitions_bootstrap, predictions[4])

        p_maes.append(p_mae)


    # Función para calcular estadísticas e intervalos de confianza (CÓDIGO NUEVO)
    def calculate_ci(maes):
        mae_promedio = np.mean(maes)
        desviacion_estandar = np.std(maes)
        valor_critico = 1.96  # Para un intervalo de confianza del 95%
        margen_de_error = valor_critico * (desviacion_estandar / np.sqrt(num_ejecuciones))
        limite_inferior = mae_promedio - margen_de_error
        limite_superior = mae_promedio + margen_de_error
        return mae_promedio, (limite_inferior, limite_superior)

    p_results = calculate_ci(p_maes)


    return {
        'p': p_results
    }

# Calcula intervalo de confianza con Bootstrap (CÓDIGO NUEVO)
resultados_bootstrap = calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, psichologicals_train,
    sequences_test, auxiliares_test, psichologicals_test,
    num_ejecuciones=1000
)

# Imprime los resultados del Bootstrap (CÓDIGO NUEVO)
for target, (mae_promedio, intervalo_confianza) in resultados_bootstrap.items():
    print(f"MAE para {target} (Regresión): {mae_promedio:.4f} (IC 95%: {intervalo_confianza})")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-17-644a453db684>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-17-644a453db684>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_15 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_27 (Dropout)      │ (None, 24)             │              0 │ lstm_15[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_15            │ (None, 27)             │              0 │ dropout_27[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 8)              │            224 │ concatenate_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_28 (Dropout)      │ (None, 8)              │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_output (Dense)          │ (None, 1)              │              9 │ dropout_28[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,977 (15.54 KB)

 Trainable params: 3,977 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 2.8047 - mae: 1.6305 - mse: 2.8047
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.4350 - mae: 1.1348 - mse: 1.4350
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5503 - mae: 0.6408 - mse: 0.5503 
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1904 - mae: 0.3521 - mse: 0.1904
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1821 - mae: 0.3509 - mse: 0.1821 
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1715 - mae: 0.3374 - mse: 0.1715
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1242 - mae: 0.2852 - mse: 0.1242
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1246 - mae: 0.2832 - mse: 0.1246
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1162 - mae: 0.2714 - mse: 0.1162
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1088 - mae: 0.2649 - mse: 0.1088
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1036 - mae: 0.255

SINGLE TASK LSTM SENSORY

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_st():
    sequential_input = Input(shape=sequential_input_shape, name='sequential_input')
    lstm_output = LSTM(units=24)(sequential_input)  # Adjust LSTM units
    lstm_output = Dropout(0.1)(lstm_output)

    auxiliary_input = Input(shape=auxiliary_input_shape, name='auxiliary_input')

    concatenated_input = Concatenate()([lstm_output, auxiliary_input])

    shared_mlp = Dense(8, activation='tanh')(concatenated_input)  # Adjust hidden units
    shared_mlp = Dropout(0.1)(shared_mlp)

    s_output = Dense(1, activation='linear', name='s_output')(shared_mlp)


    model = Model(inputs=[sequential_input, auxiliary_input], outputs=[s_output])
    return model


# Compile the model
model = create_model_st()
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss=['mse'],
              metrics=['mse','mae'])  # Adjust loss weights if needed

# Resumen del modelo
model.summary()
keras.utils.plot_model(model, 'modelo.png', show_shapes=True)

# Train the model
model.fit(x=[sequences_train, auxiliares_train], y=[sensories_train], epochs=50, batch_size=256, verbose=1, callbacks=[tensorboard_callback])

# Evaluate the model
results = model.evaluate([sequences_test, auxiliares_test],
                        [sensories_test])


# Extract loss and metrics (order should match compilation and evaluation metrics)
loss = results[0]  # Total loss
s_loss = results[1]  # Loss for 'p_output'

# Assuming correct order, retrieve MAE values
s_mae = results[2]  # MAE for 'p_output'


print(f"\nResultados de la evaluación ORIGINAL:\n"
      f"Pérdida total: {loss:.4f}\n"
      f"Pérdida para s (Regresión): {s_loss:.4f}\n"
      f"MAE para s (Regresión): {s_mae:.4f}\n"
      )


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, sensories_train,
    sequences_test, auxiliares_test, sensories_test,
    num_ejecuciones=1000
):
    s_maes = []


    for _ in range(num_ejecuciones):
        indices_bootstrap = resample(np.arange(len(sequences_test)), replace=True, n_samples=len(sequences_test))

        sequences_bootstrap = sequences_test[indices_bootstrap]
        auxiliares_bootstrap = auxiliares_test[indices_bootstrap]
        sensories_bootstrap = sensories_test[indices_bootstrap]


        # 2. Evaluate the model on the bootstrap sample (no training in each iteration)
        #predictions = model.predict([sequences_bootstrap, auxiliares_bootstrap])

         # Evaluate the model on the bootstrap sample
        results1 = model.evaluate(
        [sequences_bootstrap, auxiliares_bootstrap],
        [sensories_bootstrap],
        verbose=0  # Set verbose to 0 to suppress output during evaluation
        )

        # Extract MAE values from the results
        s_mae = results1[2]  # MAE for 's_output' (assuming the correct index)


        # Calcula el MAE para cada variable objetivo usando las predicciones (CÓDIGO NUEVO)
       # l_mae = mean_absolute_error(locomotions_bootstrap, predictions[0])
        #s_mae = mean_absolute_error(sensories_bootstrap, predictions[1])
        #v_mae = mean_absolute_error(vitalities_bootstrap, predictions[2])
        #p_mae = mean_absolute_error(psichologicals_bootstrap, predictions[3])
        #c_mae = mean_absolute_error(cognitions_bootstrap, predictions[4])

        s_maes.append(s_mae)


    # Función para calcular estadísticas e intervalos de confianza (CÓDIGO NUEVO)
    def calculate_ci(maes):
        mae_promedio = np.mean(maes)
        desviacion_estandar = np.std(maes)
        valor_critico = 1.96  # Para un intervalo de confianza del 95%
        margen_de_error = valor_critico * (desviacion_estandar / np.sqrt(num_ejecuciones))
        limite_inferior = mae_promedio - margen_de_error
        limite_superior = mae_promedio + margen_de_error
        return mae_promedio, (limite_inferior, limite_superior)

    s_results = calculate_ci(s_maes)


    return {
        's': s_results
    }

# Calcula intervalo de confianza con Bootstrap (CÓDIGO NUEVO)
resultados_bootstrap = calcular_mae_intervalo_confianza(
    sequences_train, auxiliares_train, sensories_train,
    sequences_test, auxiliares_test, sensories_test,
    num_ejecuciones=1000
)

# Imprime los resultados del Bootstrap (CÓDIGO NUEVO)
for target, (mae_promedio, intervalo_confianza) in resultados_bootstrap.items():
    print(f"MAE para {target} (Regresión): {mae_promedio:.4f} (IC 95%: {intervalo_confianza})")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-20-b8320a1bf993>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-20-b8320a1bf993>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequential_input          │ (None, 3, 14)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_18 (LSTM)            │ (None, 24)             │          3,744 │ sequential_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_33 (Dropout)      │ (None, 24)             │              0 │ lstm_18[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ auxiliary_input           │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_18            │ (None, 27)             │              0 │ dropout_33[0][0],      │
│ (Concatenate)             │                        │                │ auxiliary_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 8)              │            224 │ concatenate_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_34 (Dropout)      │ (None, 8)              │              0 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ s_output (Dense)          │ (None, 1)              │              9 │ dropout_34[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,977 (15.54 KB)

 Trainable params: 3,977 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5867 - mae: 0.6592 - mse: 0.5867
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2404 - mae: 0.4148 - mse: 0.2404
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1313 - mae: 0.3042 - mse: 0.1313
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1271 - mae: 0.2899 - mse: 0.1271 
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1086 - mae: 0.2665 - mse: 0.1086 
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0944 - mae: 0.2527 - mse: 0.0944 
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0829 - mae: 0.2361 - mse: 0.0829
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0741 - mae: 0.2210 - mse: 0.0741
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0707 - mae: 0.2131 - mse: 0.0707
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0695 - mae: 0.2134 - mse: 0.0695 
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0690 - mae: 0.2